In [1]:
import tensorflow.keras.utils as ku 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
    
data = open('/tmp/sonnets.txt').read()

--2020-06-27 20:09:00--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:400e:808::2010, 172.217.168.208, 216.58.214.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:400e:808::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.07s   

2020-06-27 20:09:00 (1.25 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
corpus = data.lower().split("\n")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences_raw = []
for line in corpus:
    seq = tokenizer.texts_to_sequences([line])
    token_list = seq[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences_raw.append(n_gram_sequence)


#

In [4]:
#pad sequences 
max_sequence_len = max([len(x) for x in input_sequences_raw])
print("max_sequence_len",max_sequence_len)
padded_seq = pad_sequences(input_sequences_raw, maxlen=max_sequence_len, padding='pre')
input_sequences = np.array(padded_seq)

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

# the labels are the next word to predict for
#label = ku.to_categorical(label, num_classes=total_words)

max_sequence_len 11


In [5]:
print(label.shape)
print(input_sequences.shape)

(15462,)
(15462, 11)


In [6]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [7]:
#model = Sequential([
#                layers.Embedding(input_dim=total_words,input_length=max_sequence_len-1, output_dim=16),
#                layers.Bidirectional(layers.LSTM(150, return_sequences = True)),
#                layers.Dropout(0.2),
#                layers.LSTM(100),
#                layers.Dense(units=total_words,activation="sigmoid")
#                ])

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(predictors,label, epochs=35, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/100
484/484 [==============================] - 19s 39ms/step - loss: 6.7908 - accuracy: 0.0243
Epoch 2/100
484/484 [==============================] - 19s 39ms/step - loss: 6.4222 - accuracy: 0.0320
Epoch 3/100
484/484 [==============================] - 20s 40ms/step - loss: 6.2131 - accuracy: 0.0359
Epoch 4/100
484/484 [==============================] - 19s 40ms/step - loss: 5.9973 - accuracy: 0.0435
Epoch 5/100
484/484 [==============================] - 20s 41ms/step - loss: 5.7315 - accuracy: 0.0505
Epoch 6/100
484/484 [==============================] - 20s 41ms/step - loss: 5.4638 - accuracy: 0.0638
Epoch 7/100
484/484 [==============================] - 20s 40ms/step - loss: 5.2033 - accuracy: 0.0782
Epoch 8/100
484/484 [==============================] - 19s 40ms/step - loss: 4.9570 - accuracy: 0.0895
Epoch 9/100
484/484 [==============================] - 19s 39ms/step - loss: 4.6902 - accuracy: 0.1060
Epoch 10/100
484/484 [==============================] - 19s 40ms/step - l